In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow(folder='logFolder'):
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./{}'.format(folder), now)
    return nowFolder

N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))

In [7]:
tf.reset_default_graph()
inpSize = X.shape[1]
layers = [10, 5, 1]
activations = [tf.tanh, tf.tanh, None]

with tf.variable_scope('io'):
    inp = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inp')
    out = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='out')
    
prevL = inpSize

with tf.variable_scope('model'):
    NN = inp * 1
    for i, (l, a) in enumerate(zip(layers, activations)):
        with tf.variable_scope('layer_{:05d}'.format(i)):
            W = tf.Variable(initial_value=np.random.rand(prevL, l), name='W', dtype=tf.float32)
            b = tf.Variable(initial_value=np.random.rand(1, l),     name='b', dtype=tf.float32)

            NN = tf.matmul(NN, W) + b
            prevL = l
        
    err = tf.reduce_mean((out - NN)**2)
        
with tf.variable_scope('optimizer'):
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(err)
    
with tf.variable_scope('misc'):
    init = tf.global_variables_initializer()
    trainableVariables = tf.trainable_variables()
    saver = tf.train.Saver(var_list = trainableVariables)


In [8]:
summaryFolder = getNow()
modelFolder   = getNow('models')

with tf.Session() as sess:
    writer = tf.summary.FileWriter(summaryFolder, sess.graph)
    sess.run(init)
    
    for i in range(1000):
        _, errVal = sess.run([opt, err], feed_dict={inp:X, out:y})
        if i % 100 == 0:
            print(errVal)
            
    os.makedirs(modelFolder)
    path = saver.save(sess, os.path.join(modelFolder, 'model.ckpt'))
    
    writer.close()
    
print('tensorboard --logdir={}'.format(summaryFolder))
print('savePath = {}'.format(path))

161.128
0.0234625
0.0147462
0.00825002
0.0043459
0.00233596
0.00141908
0.00104014
0.000895952
0.000844909
tensorboard --logdir=./logFolder/2018-06-29--17-13-20
savePath = ./models/2018-06-29--17-13-20/model.ckpt
